In [1]:
import os
import numpy as np
import tensorflow as tf
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
#from PIL import Image
import cv2 as cv
import imutils
model_file1 = "./trans_model/tf_model.pb"
model_file2 = "./trans_model/frozen_onet_model.pb"
#model_path = "./frozentensorflowModel.pb"
#pbtxt_path = "/workspace/testbuildmodel/glassdemo/dataset/glassdemo.pbtxt"



In [2]:
#testimg = "/workspace/testbuildmodel/glassdemo/dataset/Folder/lighter_video/1/out155.jpeg"
testvideo= "/workspace/testbuildmodel/glassdemo/dataset/Folder/lighter_video/1/lighter_1.mp4"

In [3]:
"""
cap = cv.VideoCapture(0)
while(True):
    print("fefe0")
  # 從攝影機擷取一張影像
    ret, frame = cap.read()
    print("fefe1")
  # 顯示圖片
    cv.imshow('frame', frame)
    print("fefe2")
  # 若按下 q 鍵則離開迴圈
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放攝影機
cap.release()

# 關閉所有 OpenCV 視窗
cv.destroyAllWindows()
"""

'\ncap = cv.VideoCapture(0)\nwhile(True):\n    print("fefe0")\n  # \xe5\xbe\x9e\xe6\x94\x9d\xe5\xbd\xb1\xe6\xa9\x9f\xe6\x93\xb7\xe5\x8f\x96\xe4\xb8\x80\xe5\xbc\xb5\xe5\xbd\xb1\xe5\x83\x8f\n    ret, frame = cap.read()\n    print("fefe1")\n  # \xe9\xa1\xaf\xe7\xa4\xba\xe5\x9c\x96\xe7\x89\x87\n    cv.imshow(\'frame\', frame)\n    print("fefe2")\n  # \xe8\x8b\xa5\xe6\x8c\x89\xe4\xb8\x8b q \xe9\x8d\xb5\xe5\x89\x87\xe9\x9b\xa2\xe9\x96\x8b\xe8\xbf\xb4\xe5\x9c\x88\n    if cv.waitKey(1) & 0xFF == ord(\'q\'):\n        break\n\n# \xe9\x87\x8b\xe6\x94\xbe\xe6\x94\x9d\xe5\xbd\xb1\xe6\xa9\x9f\ncap.release()\n\n# \xe9\x97\x9c\xe9\x96\x89\xe6\x89\x80\xe6\x9c\x89 OpenCV \xe8\xa6\x96\xe7\xaa\x97\ncv.destroyAllWindows()\n'

In [4]:
"""
with tf.gfile.FastGFile(model_path, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Session() as sess:
    # Restore session
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
"""

"\nwith tf.gfile.FastGFile(model_path, 'rb') as f:\n    graph_def = tf.GraphDef()\n    graph_def.ParseFromString(f.read())\n\nwith tf.Session() as sess:\n    # Restore session\n    sess.graph.as_default()\n    tf.import_graph_def(graph_def, name='')\n"

In [5]:
def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

In [6]:
classify_graph1 = load_graph(model_file1)
sess1=tf.Session(graph=classify_graph1)

classify_graph2 = load_graph(model_file2)
sess2=tf.Session(graph=classify_graph2)

In [7]:
#for op in tf.get_default_graph().get_operations():
    #print(str(op.name))

In [8]:
#sess2.graph.get_operations()

In [9]:
"""
import time

with tf.Session() as sess:
    # Restore session

    #alltensor=sess.graph.get_operations()
    #print(alltensor)
    cap = cv.VideoCapture(0)
    cap.set(cv.CAP_PROP_FRAME_WIDTH, 1280);
    cap.set(cv.CAP_PROP_FRAME_HEIGHT, 720);
    while(True):
        
       
        ret, img = cap.read()
        # Read and preprocess an image.
        #img = cv.imread(testimg)
        rows = img.shape[0]
        cols = img.shape[1]
        img2=img.copy()
        inp = cv.resize(img2, (300, 300))
        #inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

        # Run the model
        start_time = time.time()
        print("FFFFF")
        out = sess.run([sess.graph.get_tensor_by_name('claasf1_1/dense_1/BiasAdd:0'),],
                       feed_dict={'Reshape:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})
        print("FFFFF")
        elapsed_time = time.time() - start_time
        print(elapsed_time)
        predctedArray=out[0][0].astype(int)
        
        # Visualize detected bounding boxes.

        x1=max(0, min(predctedArray[2]-179, 319))
        if(x1!=319):
            y1=max(0, min(predctedArray[2]-x1, 179))
        else:
            y1=max(0, min(179-(predctedArray[2]-319-179), 179))

        x2=max(0, min(predctedArray[3]-179, 319))
        if(x2!=319):
            y2=max(0, min(predctedArray[3]-x1, 179))
        else:
            y2=max(0, min(179-(predctedArray[2]-319-179), 179))  
        #print(rows)
        #print(cols)
        img=cv.resize(img, (320, 180))
        cv.line(img, (predctedArray[0],predctedArray[1]), ( x1,y1), (0,0,255), thickness=1, lineType=8,  shift=0)
        cv.line(img, (predctedArray[0],predctedArray[1]), ( x2,y2),(0,0,255), thickness=1, lineType=8,  shift=0)

        #cv.putText(img, text, (10, 80), cv.FONT_HERSHEY_PLAIN,1, (0, 255, 255), 1, cv.LINE_AA)

                #print(classId, "-->", score, x, y)
        cv.imshow('frame', img)

        if cv.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()


cv.destroyAllWindows()
#cv.imshow('SHOWfg', imutils.resize(img, width=800))
#cv.waitKey()
"""

'\nimport time\n\nwith tf.Session() as sess:\n    # Restore session\n\n    #alltensor=sess.graph.get_operations()\n    #print(alltensor)\n    cap = cv.VideoCapture(0)\n    cap.set(cv.CAP_PROP_FRAME_WIDTH, 1280);\n    cap.set(cv.CAP_PROP_FRAME_HEIGHT, 720);\n    while(True):\n        \n       \n        ret, img = cap.read()\n        # Read and preprocess an image.\n        #img = cv.imread(testimg)\n        rows = img.shape[0]\n        cols = img.shape[1]\n        img2=img.copy()\n        inp = cv.resize(img2, (300, 300))\n        #inp = inp[:, :, [2, 1, 0]]  # BGR2RGB\n\n        # Run the model\n        start_time = time.time()\n        print("FFFFF")\n        out = sess.run([sess.graph.get_tensor_by_name(\'claasf1_1/dense_1/BiasAdd:0\'),],\n                       feed_dict={\'Reshape:0\': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})\n        print("FFFFF")\n        elapsed_time = time.time() - start_time\n        print(elapsed_time)\n        predctedArray=out[0][0].astype(int)\n   

In [10]:
#input_1_2:0
#dense_1_2/BiasAdd:0


In [11]:
import time


    # Restore session

    #alltensor=sess.graph.get_operations()
    #print(alltensor)
cap = cv.VideoCapture(0)
cap.set(cv.CAP_PROP_FRAME_WIDTH, 1280);
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 720);
while(True):

    print(cap)
    ret, img = cap.read()
    # Read and preprocess an image.
    #img = cv.imread(testimg)
    rows = img.shape[0]
    cols = img.shape[1]
    img2=img.copy()
    midpint=cols/2
    print (img2.shape)
    img2 = img2[:, midpint-rows/2:midpint+rows/2,:]
    print(img2.shape)
    inp = cv.resize(img2, (100, 100))
    inp2 = cv.resize(img2, (48, 48))
    #inp2 = inp2[:, :, [2, 1, 0]]
    inp2=(inp2 - 127.5) * 0.0078125
    #print(in2p)
    #inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

    # Run the model
    start_time = time.time()

    out2 = sess2.run([sess2.graph.get_tensor_by_name('import/onet/conv6-2/onet/conv6-2:0'),sess2.graph.get_tensor_by_name('import/onet/conv6-3/onet/conv6-3:0')],
       feed_dict={'import/Placeholder:0': inp2.reshape(1, inp2.shape[0], inp2.shape[1], 3)})
    elapsed_time1 = time.time() - start_time
    
    start_time = time.time()
    out1 = sess1.run([sess1.graph.get_tensor_by_name('import/dense_1/BiasAdd:0'),],
                   feed_dict={'import/input_1:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})
    



    elapsed_time2 = time.time() - start_time
    print(elapsed_time1)
    print(elapsed_time2)
    predctedArray=out1[0][0]
    print('############')
    predctedArray2=out2[1]
    predctedArray2=(predctedArray2+1)/2
    predctedArray2=predctedArray2[0]
    print(predctedArray2.shape)
    print(predctedArray2)
    print('############')
    # Visualize detected bounding boxes.


    #print(predctedArray)
    #print(rows)
    #print(cols)
    img=cv.resize(img2, (300, 300))
    cv.ellipse(img, (int(predctedArray[0]*300),int(predctedArray[1]*300)), (20, 20), 0, 0, 0, (205, 0, 255), 10)
    cv.ellipse(img, (int(predctedArray[2]*300),int(predctedArray[3]*300)), (20, 20), 0, 0, 0, (205, 0, 255), 10)
    cv.ellipse(img, (int(predctedArray[4]*300),int(predctedArray[5]*300)), (20, 20), 0, 0, 0, (205, 0, 255), 10)
    cv.ellipse(img, (int(predctedArray[6]*300),int(predctedArray[7]*300)), (20, 20), 0, 0, 0, (205, 0, 255), 10)
    cv.ellipse(img, (int(predctedArray[8]*300),int(predctedArray[9]*300)), (20, 20), 0, 0, 0, (205, 0, 255), 10)
   
    cv.ellipse(img, (int(predctedArray2[0]*300),int(predctedArray2[1]*300)), (20, 20), 0, 0, 0, (205, 0, 0), 10)
    cv.ellipse(img, (int(predctedArray2[2]*300),int(predctedArray2[3]*300)), (20, 20), 0, 0, 0, (205, 0, 0), 10)
    cv.ellipse(img, (int(predctedArray2[4]*300),int(predctedArray2[5]*300)), (20, 20), 0, 0, 0, (205, 0, 0), 10)
    cv.ellipse(img, (int(predctedArray2[6]*300),int(predctedArray2[7]*300)), (20, 20), 0, 0, 0, (205, 0, 0), 10)
    cv.ellipse(img, (int(predctedArray2[8]*300),int(predctedArray2[9]*300)), (20, 20), 0, 0, 0, (205, 0, 0), 10)
    #cv.putText(img, text, (10, 80), cv.FONT_HERSHEY_PLAIN,1, (0, 255, 255), 1, cv.LINE_AA)

            #print(classId, "-->", score, x, y)
    cv.imshow('frame', img)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()


cv.destroyAllWindows()
#cv.imshow('SHOWfg', imutils.resize(img, width=800))
#cv.waitKey()

<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.622210025787
1.12283587456
############
(10,)
[0.33080053 0.22105509 0.6617409  0.22859299 0.39490876 0.40052184
 0.306871   0.7356151  0.5738483  0.7466111 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00401210784912
0.00162482261658
############
(10,)
[0.33667165 0.18896061 0.67058057 0.2077412  0.4052856  0.3700811
 0.31601685 0.7041579  0.57719254 0.71739864]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00200510025024
0.00126791000366
############
(10,)
[0.34610927 0.19248879 0.684055   0.21649808 0.41624984 0.37601316
 0.30624658 0.7107059  0.5722316  0.73424065]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00148701667786
0.00088906288147
############
(10,)
[0.3391167  0.19930506 0.68335366 0.21896034 0.41997433 0.3706137
 0.29872888 0.7098865  0.574513   0.7348145 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720,

(720, 1280, 3)
(720, 720, 3)
0.00171399116516
0.000851154327393
############
(10,)
[0.36252868 0.4013314  0.59198666 0.39261413 0.33478677 0.56735265
 0.3194     0.77209914 0.48374218 0.74269766]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00148797035217
0.00082802772522
############
(10,)
[0.39627    0.3932532  0.508943   0.39318788 0.32781476 0.4923315
 0.3776937  0.77047896 0.38068908 0.7064961 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00197696685791
0.00104093551636
############
(10,)
[0.34510916 0.36459765 0.5813069  0.33967173 0.40978694 0.5061821
 0.38906795 0.70095783 0.55270165 0.63596094]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00210785865784
0.00113797187805
############
(10,)
[0.41852832 0.43285486 0.61213225 0.3164764  0.57117176 0.4816026
 0.5890687  0.7345699  0.6461897  0.6298568 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00197219848633
0.00

0.00780606269836
0.00395607948303
############
(10,)
[0.47052798 0.34791154 0.6953629  0.33553624 0.639733   0.57592803
 0.50566244 0.7489878  0.6353363  0.7104781 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00630903244019
0.00347399711609
############
(10,)
[0.43134913 0.3474422  0.6769245  0.35709125 0.6013606  0.5549813
 0.45647442 0.71086735 0.59781075 0.7012111 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00647401809692
0.00339889526367
############
(10,)
[0.4111693  0.32693213 0.6604966  0.35352862 0.56061393 0.55019397
 0.4465384  0.7065839  0.57856834 0.7048862 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00273203849792
0.00159215927124
############
(10,)
[0.46899718 0.32404882 0.7154406  0.33833325 0.64140505 0.52089506
 0.4950406  0.7133251  0.64387274 0.6948689 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00659704208374
0.00348806381226
############
(1

(720, 1280, 3)
(720, 720, 3)
0.00237512588501
0.00118803977966
############
(10,)
[0.49476242 0.38841817 0.6829057  0.35642093 0.6487361  0.54679525
 0.5325599  0.7488747  0.62866735 0.7142941 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00736498832703
0.00353312492371
############
(10,)
[0.45475444 0.34121692 0.7164524  0.3045336  0.64889336 0.5057718
 0.51615435 0.6884147  0.6658481  0.6733668 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00698304176331
0.00385594367981
############
(10,)
[0.44680214 0.38219136 0.67854947 0.40012053 0.6176891  0.5671191
 0.5198176  0.74637824 0.60211456 0.7266133 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.0044059753418
0.00242400169373
############
(10,)
[0.44710162 0.32686353 0.675116   0.27732655 0.62437534 0.47285658
 0.54562825 0.6851944  0.65348583 0.6329963 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00634407997131
0.002

(10,)
[0.44444412 0.35847566 0.7189043  0.32843238 0.6000551  0.5254709
 0.5162261  0.65205324 0.7099013  0.6223465 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00207805633545
0.00106000900269
############
(10,)
[0.43437874 0.35934997 0.72664183 0.3325777  0.6002227  0.530885
 0.51412696 0.6659958  0.717511   0.63776505]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00194692611694
0.00138211250305
############
(10,)
[0.47647893 0.35371682 0.7551775  0.33828542 0.6035516  0.52930903
 0.5219662  0.645807   0.72146106 0.63960326]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00683403015137
0.00362491607666
############
(10,)
[0.45648926 0.35979897 0.76939654 0.33738774 0.61064696 0.5309727
 0.51870483 0.67367786 0.74055153 0.6607251 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00681090354919
0.00353407859802
############
(10,)
[0.46209818 0.35382932 0.77563596 0.34167853 0

...

<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00183296203613
0.00107598304749
############
(10,)
[0.36113203 0.3706216  0.8178529  0.34355086 0.58637697 0.63982713
 0.45435667 0.8401967  0.7639566  0.8243004 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00640988349915
0.00347900390625
############
(10,)
[0.35496575 0.37151292 0.81513053 0.34375405 0.58226615 0.6401465
 0.4489211  0.83449185 0.7643089  0.81795967]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00181603431702
0.00117588043213
############
(10,)
[0.36159223 0.36846375 0.8190737  0.3443362  0.5804538  0.64234173
 0.4511745  0.8325979  0.7648819  0.81889975]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00667595863342
0.00353598594666
############
(10,)
[0.36089337 0.36973155 0.8200668  0.3401997  0.5858773  0.64114374
 0.45542303 0.84157777 0.76675856 0.8236735 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720

<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00387096405029
0.00205397605896
############
(10,)
[0.38878205 0.3548351  0.838432   0.32800543 0.60677236 0.6248143
 0.47872114 0.821527   0.7876641  0.8082537 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00675201416016
0.00326895713806
############
(10,)
[0.4017211  0.34815878 0.84258085 0.32939783 0.5927521  0.6299908
 0.47995722 0.82125986 0.7817086  0.81361294]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00427412986755
0.00250196456909
############
(10,)
[0.40799645 0.34575862 0.84783906 0.32755798 0.58923763 0.6350821
 0.48355174 0.8216928  0.7851747  0.8158176 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00643110275269
0.00340914726257
############
(10,)
[0.39079607 0.3596511  0.82818973 0.32987815 0.5955363  0.63273054
 0.4784642  0.82426196 0.7821574  0.8099822 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 

0.00690317153931
0.00376009941101
############
(10,)
[0.28762892 0.28356862 0.66172    0.26941615 0.3594572  0.5108915
 0.31614506 0.7345055  0.5766049  0.73105717]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00351214408875
0.00189208984375
############
(10,)
[0.2656219  0.292721   0.6624551  0.27829483 0.34808198 0.51734984
 0.29280907 0.74000704 0.5783973  0.7384267 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00595307350159
0.00346493721008
############
(10,)
[0.27030978 0.2956744  0.6618729  0.27922422 0.34378988 0.5238213
 0.30040085 0.74233127 0.5811466  0.7400947 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.0068531036377
0.00419187545776
############
(10,)
[0.27806085 0.28994852 0.6634421  0.27205664 0.3594687  0.5198409
 0.31274056 0.7487432  0.5860751  0.7427393 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00670504570007
0.00341105461121
############
(10,)

<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00325894355774
0.00167012214661
############
(10,)
[0.2898231  0.3932942  0.6981422  0.36561024 0.43936783 0.642431
 0.3779984  0.8290968  0.66753435 0.81642556]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00633788108826
0.00343012809753
############
(10,)
[0.28993845 0.39325655 0.69620574 0.37079757 0.44164258 0.63878614
 0.36775312 0.8269322  0.6591313  0.8194553 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.0070948600769
0.00322794914246
############
(10,)
[0.2951548  0.3979286  0.6887518  0.3885149  0.42774618 0.64220214
 0.35670263 0.81830335 0.64595026 0.82063246]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00649881362915
0.00341105461121
############
(10,)
[0.27578887 0.40578222 0.68376774 0.38882858 0.4325839  0.64582276
 0.34191743 0.82397175 0.64017475 0.81900644]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 

0.00726294517517
0.00363492965698
############
(10,)
[0.27868432 0.4174413  0.7043158  0.38176426 0.46738738 0.62697726
 0.38686797 0.811268   0.67907053 0.7892362 ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00331902503967
0.00192403793335
############
(10,)
[0.5566596  0.40865564 0.9160377  0.36060405 0.8719314  0.6062439
 0.66722876 0.82979035 0.8789563  0.774912  ]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00665307044983
0.00411415100098
############
(10,)
[0.2387129  0.29611245 0.6487022  0.28108484 0.43260714 0.4968919
 0.26735392 0.75240237 0.5518761  0.72909933]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00736212730408
0.00349497795105
############
(10,)
[0.40667224 0.32460466 0.80222815 0.35651237 0.52140236 0.53541064
 0.3810421  0.80378735 0.6053435  0.77046317]
############
<VideoCapture 0x7f16c06456f0>
(720, 1280, 3)
(720, 720, 3)
0.00689196586609
0.00343179702759
############
(10